Jupter utilizando puramente a RNN sem a aplicação de uma PCA antes do processo

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

Ler os dados dos csv com os dados simulados contidos nas pastas

In [8]:
def ler_csv_e_atribuir_classes_com_pca(diretorio, classe, n_components=5):
    dados = []
    for arquivo in os.listdir(diretorio):
        if arquivo.endswith('.csv'):
            caminho_arquivo = os.path.join(diretorio, arquivo)
            df = pd.read_csv(caminho_arquivo, sep=';')
            features = df.iloc[:, :-1]  
            features_normalizadas = (features - features.mean()) / features.std()  # Normalização
            df_reduzido = pd.DataFrame(data=features_normalizadas, columns=[f'componente_{i}' for i in range(1, n_components + 1)])
            df_reduzido['Classe'] = classe
            dados.append(df_reduzido)
    return pd.concat(dados, ignore_index=True)

diretorio_antes = './64mics/antes'
diretorio_depois = './64mics/depois'

dados_antes = ler_csv_e_atribuir_classes_com_pca(diretorio_antes, classe=0)

dados_depois = ler_csv_e_atribuir_classes_com_pca(diretorio_depois, classe=1)

dados_combinados = pd.concat([dados_antes, dados_depois], ignore_index=True)

In [9]:
dados_combinados.shape

(11000000, 6)

In [10]:
import numpy as np

def split_into_sequences(data, sequence_length=10000):
    sequences = []
    labels = []
    for i in range(0, len(data), sequence_length):
        sequence = data.iloc[i:i+sequence_length, :-1]  
        label = data.iloc[i+sequence_length-1, -1]  
        sequences.append(sequence.values)
        labels.append(label)
    return np.array(sequences), np.array(labels)

X_sequences, y_sequences = split_into_sequences(dados_combinados)

print("Dimensões das sequências de características:", X_sequences.shape)
print("Dimensões das sequências de rótulos:", y_sequences.shape)


Dimensões das sequências de características: (1100, 10000, 5)
Dimensões das sequências de rótulos: (1100,)


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

print("Dimensões dos dados de treinamento de características:", X_train.shape)
print("Dimensões dos dados de teste de características:", X_test.shape)
print("Dimensões dos dados de treinamento de rótulos:", y_train.shape)
print("Dimensões dos dados de teste de rótulos:", y_test.shape)

Dimensões dos dados de treinamento de características: (880, 10000, 5)
Dimensões dos dados de teste de características: (220, 10000, 5)
Dimensões dos dados de treinamento de rótulos: (880,)
Dimensões dos dados de teste de rótulos: (220,)


Realização do treinamento

In [12]:
checkpoint_path = "model_checkpoint_ger_test.h5"

checkpoint_callback = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2, callbacks=[checkpoint_callback])

Epoch 1/5
11/11 [==============================] - ETA: 0s - loss: nan - accuracy: 0.4801 
Epoch 1: val_accuracy improved from -inf to 0.44886, saving model to model_checkpoint_ger_test.h5
11/11 [==============================] - 200s 18s/step - loss: nan - accuracy: 0.4801 - val_loss: nan - val_accuracy: 0.4489
Epoch 2/5


c:\Users\Gabe\anaconda3\envs\google-colab-ver\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


11/11 [==============================] - ETA: 0s - loss: nan - accuracy: 0.4801 
Epoch 2: val_accuracy did not improve from 0.44886
11/11 [==============================] - 205s 19s/step - loss: nan - accuracy: 0.4801 - val_loss: nan - val_accuracy: 0.4489
Epoch 3/5
11/11 [==============================] - ETA: 0s - loss: nan - accuracy: 0.4801 
Epoch 3: val_accuracy did not improve from 0.44886
11/11 [==============================] - 206s 19s/step - loss: nan - accuracy: 0.4801 - val_loss: nan - val_accuracy: 0.4489
Epoch 4/5
11/11 [==============================] - ETA: 0s - loss: nan - accuracy: 0.4801 
Epoch 4: val_accuracy did not improve from 0.44886
11/11 [==============================] - 206s 19s/step - loss: nan - accuracy: 0.4801 - val_loss: nan - val_accuracy: 0.4489
Epoch 5/5
11/11 [==============================] - ETA: 0s - loss: nan - accuracy: 0.4801 
Epoch 5: val_accuracy did not improve from 0.44886
11/11 [==============================] - 207s 19s/step - loss: nan 

In [13]:
import h5py

with h5py.File(checkpoint_path, 'r') as f:
    best_model = tf.keras.models.load_model(f)


In [14]:
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f'Acurácia do melhor modelo: {accuracy}')

7/7 [==============================] - 7s 981ms/step - loss: nan - accuracy: 0.5045
Acurácia do melhor modelo: 0.5045454502105713
